In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import pandas as pd
import os
import seaborn as sns
import numpy as np
import tensorflow.keras.backend as kb
import ml_util_4_head
import time
from scipy import signal
import importlib
from tensorflow.keras.losses import BinaryCrossentropy, MeanAbsoluteError, MeanSquaredError

In [2]:
%matplotlib tk 

In [4]:
#Experiment_options
importlib.reload(ml_util_4_head)
window_size = 200
num_snapshots_in_sequence = 300
use_smooth = False
do_filter = False
predict_future_by = 0 # num samples to predict into the future
use_side_flag = False
subjects_to_train_with = [1,2,3,4,5,6,7,8,9]



sides = ['LEFT', 'RIGHT']

trial_nums = 1+np.arange(11)
print('trials to train with: ', trial_nums, 'subjects to train with: ', subjects_to_train_with)

root_folder = "unilateral_4_head"
print(root_folder)
sequence_len = num_snapshots_in_sequence + window_size - 1
training_instances = np.empty(shape=[0,sequence_len, 12], dtype=np.float32)
files_to_train_with = ml_util_4_head.get_files_to_use(root_folder, subjects_to_train_with, sides, trial_nums)


trials to train with:  [ 1  2  3  4  5  6  7  8  9 10 11] subjects to train with:  [1, 2, 3, 4, 5, 6, 7, 8, 9]
unilateral_4_head


In [5]:
def custom_loss(y_actual, y_pred):
    mask = kb.greater_equal(y_actual, 0)
    mask = tf.cast(mask, tf.float32)
    custom_loss = tf.math.reduce_sum(
        kb.square(mask*(y_actual-y_pred)))/tf.math.reduce_sum(mask)
    return custom_loss

In [6]:
def custom_loss_for_ramp(y_actual, y_pred):
    # Create a mask where `y_actual` is not equal to -100
    mask = kb.not_equal(y_actual, -100)
    # Cast the mask to float32
    mask = tf.cast(mask, tf.float32)
    # Use the mask to ignore the values where `y_actual` is -100
    masked_squared_error = kb.square(mask * (y_actual - y_pred))
    # Calculate the sum of the squared errors where the mask is True
    numerator = tf.math.reduce_sum(masked_squared_error)
    # Calculate the sum of the mask values (essentially the count of non-masked elements)
    denominator = tf.math.reduce_sum(mask)
    # To avoid division by zero, add a small constant to the denominator
    denominator = tf.where(tf.equal(denominator, 0), tf.constant(1, dtype=tf.float32), denominator)
    # Compute the mean squared error while ignoring the masked values
    custom_loss_value = numerator / denominator
    return custom_loss_value

In [7]:
def manipulate_ss_col(ss_col):
    flag = 0  # Flags: 0 -> looking for first positive, 1 -> searching for 1, 2 -> holding 1, 3 -> setting to -1, 4 -> holding 0
    counter = 0
    first_positive_found = False  # Indicator for the first positive value

    for i in range(len(ss_col)):
        if not first_positive_found:
            if ss_col[i] > 0:
                first_positive_found = True
                flag = 1  # Start looking for the next 1
            else:
                ss_col[i] = -1  # Set to -1 until the first positive value is found
        else:
            if flag == 1 and ss_col[i] == 1:
                # When 1 is found, start holding at 1
                flag = 2
                counter = 1
            elif flag == 2:
                # Hold 1 for 15 points
                if counter < 15:
                    ss_col[i] = 1
                    counter += 1
                else:
                    # Then go to -1 for the next 10 points
                    flag = 3
                    counter = 1
                    ss_col[i] = -1
            elif flag == 3:
                # Hold -1 for 10 points
                if counter < 10:
                    ss_col[i] = -1
                    counter += 1
                else:
                    # Then go back to 0 and start looking for 1 again
                    flag = 4
                    counter = 0
            elif flag == 4 and ss_col[i] != 0:
                # As soon as the value starts increasing, start looking for 1 again
                flag = 1
    
    return ss_col

In [8]:
def construct_model_2023v2(window_size,
                                      filter_sizes,
                                      kernel_sizes,
                                      dilations,
                                      num_channels=8,
                                      batch_norm_insertion_pts=[2],
                                      sp_dense_sizes=[20, 10],
                                      ss_dense_sizes=[20, 10],
                                      v_dense_sizes=[20, 10],
                                      r_dense_sizes=[20, 10],
                                      do_fix_input_dim=False):
    if len(filter_sizes) != len(kernel_sizes)+1:
        raise ValueError(
            'Must provide one more filter size than kernel size--last kernel size is calculated')
    current_output_size = window_size  # Track for final conv layer

    #Use None in dim 0 to allow variable input length.
    #Use window_size to fix size--helpful for debugging dimensions
    if do_fix_input_dim:
        input_layer = tf.keras.layers.Input(
            shape=(window_size, num_channels), name='my_input_layer')
    else:
        input_layer = tf.keras.layers.Input(
            shape=(None, num_channels), name='my_input_layer')

    # outputs = []
    z = input_layer
    # Conv1D for each head
    for layer_idx in range(len(kernel_sizes)):
        z = tf.keras.layers.Conv1D(filters=filter_sizes[layer_idx], kernel_size=kernel_sizes[layer_idx],
                                    dilation_rate=dilations[layer_idx], activation='relu')(z)
        if layer_idx in batch_norm_insertion_pts:
            z = tf.keras.layers.BatchNormalization()(z)
        current_output_size = current_output_size - \
            dilations[layer_idx]*kernel_sizes[layer_idx] + dilations[layer_idx]
    if current_output_size < 1:
        raise ValueError('layers shrink the cnn too much')
    else:
        print('adding final conv layer of kernel size: ', current_output_size)
        z = tf.keras.layers.Conv1D(
            filters=filter_sizes[-1], kernel_size=current_output_size, activation='relu')(z)
    for num_neurons in sp_dense_sizes:
        z = tf.keras.layers.Dense(num_neurons, activation='relu')(z)
    output_stance_phase = tf.keras.layers.Dense(1, name='stance_phase_output')(z)    


    current_output_size = window_size      
    z = input_layer
    # Conv1D for each head
    for layer_idx in range(len(kernel_sizes)):
        z = tf.keras.layers.Conv1D(filters=filter_sizes[layer_idx], kernel_size=kernel_sizes[layer_idx],
                                    dilation_rate=dilations[layer_idx], activation='relu')(z)
        if layer_idx in batch_norm_insertion_pts:
            z = tf.keras.layers.BatchNormalization()(z)
        current_output_size = current_output_size - \
            dilations[layer_idx]*kernel_sizes[layer_idx] + dilations[layer_idx]
    if current_output_size < 1:
        raise ValueError('layers shrink the cnn too much')
    else:
        print('adding final conv layer of kernel size: ', current_output_size)
        z = tf.keras.layers.Conv1D(
            filters=filter_sizes[-1], kernel_size=current_output_size, activation='relu')(z)
    for num_neurons in ss_dense_sizes:
        z = tf.keras.layers.Dense(num_neurons, activation='relu')(z)
    output_stance_swing = tf.keras.layers.Dense(
        1, activation='sigmoid', name='stance_swing_output')(z)
    
    current_output_size = window_size
    z = input_layer
    # Conv1D for each head
    for layer_idx in range(len(kernel_sizes)):
        z = tf.keras.layers.Conv1D(filters=filter_sizes[layer_idx], kernel_size=kernel_sizes[layer_idx],
                                    dilation_rate=dilations[layer_idx], activation='relu')(z)
        if layer_idx in batch_norm_insertion_pts:
            z = tf.keras.layers.BatchNormalization()(z)
        current_output_size = current_output_size - \
            dilations[layer_idx]*kernel_sizes[layer_idx] + dilations[layer_idx]
    if current_output_size < 1:
        raise ValueError('layers shrink the cnn too much')
    else:
        print('adding final conv layer of kernel size: ', current_output_size)
        z = tf.keras.layers.Conv1D(
            filters=filter_sizes[-1], kernel_size=current_output_size, activation='relu')(z)
    for num_neurons in v_dense_sizes:
        z = tf.keras.layers.Dense(num_neurons, activation='relu')(z)
    velocity = tf.keras.layers.Dense(1, name='velocity_output')(z) 

    z = input_layer
    current_output_size = window_size
    # Conv1D for each head
    for layer_idx in range(len(kernel_sizes)):
        z = tf.keras.layers.Conv1D(filters=filter_sizes[layer_idx], kernel_size=kernel_sizes[layer_idx],
                                    dilation_rate=dilations[layer_idx], activation='relu')(z)
        if layer_idx in batch_norm_insertion_pts:
            z = tf.keras.layers.BatchNormalization()(z)
        current_output_size = current_output_size - \
            dilations[layer_idx]*kernel_sizes[layer_idx] + dilations[layer_idx]
    if current_output_size < 1:
        raise ValueError('layers shrink the cnn too much')
    else:
        print('adding final conv layer of kernel size: ', current_output_size)
        z = tf.keras.layers.Conv1D(
            filters=filter_sizes[-1], kernel_size=current_output_size, activation='relu')(z)
    for num_neurons in r_dense_sizes:
        z = tf.keras.layers.Dense(num_neurons, activation='relu')(z)
    ramp = tf.keras.layers.Dense(1, name='ramp_output')(z)  

    model = tf.keras.Model(inputs=[input_layer], outputs=[
    output_stance_phase, output_stance_swing, velocity, ramp])

    return model


In [9]:
for myfile in files_to_train_with:
    data = ml_util_4_head.load_file(myfile)
    
    ss_col = data[:,-4]
    ss_col = manipulate_ss_col(ss_col)
    data[:,-4] = ss_col
    # MAX 
    ramp_col = data[:,-2]
    ramp_col[ss_col==0]=-100
    data[:,-2] = ramp_col
    num_rows, num_cols = data.shape
    num_rows_to_drop = num_rows % sequence_len
    data = data[0:-num_rows_to_drop]
    new_num_rows, num_cols = data.shape
    num_sequences = new_num_rows/sequence_len
    new_data_shape = (int(num_sequences), sequence_len, num_cols)
    new_instances = data.reshape(new_data_shape)
    training_instances = np.append(training_instances, new_instances, axis=0)

shuffled_training_instances = tf.random.shuffle(training_instances) 
num_channels = 8
x = shuffled_training_instances[:, :, :num_channels]
y_v = shuffled_training_instances[:, window_size-1:,-1]
y_r = shuffled_training_instances[:, window_size-1:,-2]
y_sp = shuffled_training_instances[:, window_size-1:,-4]
y_ss = shuffled_training_instances[:, window_size-1:,-3]


split_fraction = 0.8
split_num = int(np.rint(split_fraction*x.shape[0]))
x_train = x[:split_num,:,:]
x_train = tf.cast(x_train, tf.float32)
y_sp_train = y_sp[:split_num,:]
y_ss_train = y_ss[:split_num,:]
y_r_train = y_r[:split_num,:]
y_v_train = y_v[:split_num,:]

x_valid = x[split_num:,:,:]
y_sp_valid = y_sp[split_num:,:]
y_ss_valid = y_ss[split_num:,:]
y_r_valid = y_r[split_num:,:]
y_v_valid = y_v[split_num:,:]


print('shape of x_train: ', x_train.shape, 'shape of y_gp_train: ', y_sp_train.shape, 'shape of y_ss_train: ', y_ss_train.shape,
'\nshape of x_valid: ', x_valid.shape, 'shape of y_gp_valid: ', y_sp_valid.shape, 'shape of y_ss_valid: ', y_ss_valid.shape)
print(x_train.dtype)



shape of x_train:  (6402, 499, 8) shape of y_gp_train:  (6402, 300) shape of y_ss_train:  (6402, 300) 
shape of x_valid:  (1600, 499, 8) shape of y_gp_valid:  (1600, 300) shape of y_ss_valid:  (1600, 300)
<dtype: 'float32'>


In [ ]:
plt.plot(np.transpose(y_v_valid))
plt.show()

In [ ]:
plt.plot(np.transpose(y_sp_valid))
plt.show()

In [ ]:
plt.plot(np.transpose(y_ss_valid))
plt.show()

In [ ]:
plt.plot(np.transpose(y_r_valid))
plt.show()

In [11]:
# # BUILD MODEL
importlib.reload(ml_util_4_head)
num_channels = 8
do_fix_input_dim=False

tf.keras.backend.clear_session()

model = construct_model_2023v2(window_size=window_size,
    filter_sizes=[44,44,44,44], kernel_sizes=[8,8,8], dilations=[6,6,6], num_channels=8,
    batch_norm_insertion_pts=[0,1], sp_dense_sizes=[23,23,23,23], ss_dense_sizes=[23,23,23,23], v_dense_sizes=[23,23,23,23], r_dense_sizes=[23,23,23,23], do_fix_input_dim=do_fix_input_dim)

print(model.summary())
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss=[ml_util_4_head.custom_loss, 'binary_crossentropy', ml_util_4_head.custom_loss, custom_loss_for_ramp], loss_weights=[4,1,0.3,0.1], optimizer=optimizer)

adding final conv layer of kernel size:  74
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 my_input_layer (InputLayer  [(None, None, 8)]            0         []                            
 )                                                                                                
                                                                                                  
 conv1d (Conv1D)             (None, None, 44)             2860      ['my_input_layer[0][0]']      
                                                                                                  
 batch_normalization (Batch  (None, None, 44)             176       ['conv1d[0][0]']              
 Normalization)                                                                                   
                                                  

In [12]:
filename = "test.h5"

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, restore_best_weights=True)
mc = tf.keras.callbacks.ModelCheckpoint( filename, monitor='val_loss', mode='min', save_best_only=True, verbose=1)
history = model.fit(x=x_train, y=[y_sp_train, y_ss_train, y_v_train, y_r_train], 
    batch_size=32, epochs=50, validation_data=(x_valid,[y_sp_valid, y_ss_valid, y_v_valid, y_r_valid]), 
    callbacks=[es, mc], verbose=1)

In [14]:
model = tf.keras.models.load_model('letssee.h5', custom_objects={'custom_loss': custom_loss, 'custom_loss_for_ramp' : custom_loss_for_ramp})


In [15]:
subjects_to_test_with = [10]
trial_nums = [1,2,5,6]
print('trials to test with: ', trial_nums, 'subjects to train with: ', subjects_to_test_with)
sides = ['RIGHT', 'LEFT']

root_folder = "unilateral_4_head"
print(root_folder)

files_to_test_with = ml_util_4_head.get_files_to_use(root_folder, subjects_to_test_with, sides, trial_nums)


trials to test with:  [1, 2, 5, 6] subjects to train with:  [10]
unilateral_4_head


In [16]:
rmses = []
rmses_vel = []
rmses_isp = []
rmses_ramp = []



num_channels = 8

for myfile in files_to_test_with:
    data = ml_util_4_head.load_file(myfile)

    ss_col_test = data[window_size-1:,-4]


    # MAX 
    ramp_col_test = data[window_size-1:,-2]
    ramp_col_test[ss_col_test==0]=-100
    data[window_size-1:,-2] = ramp_col_test



    x_test = tf.expand_dims(data[:,:num_channels], axis=0)
    x_test=tf.cast(x_test, dtype = tf.float32)
    y_sp_test = data[window_size-1:,-4]
    y_ss_test = data[window_size-1:,-3]
    y_v_test = data[window_size-1:,-1]
    y_r_test = data[window_size-1:,-2]


    model_outputs = model.predict(x=x_test)    
    y_sp_predict, y_ss_predict, y_v_predict, y_r_predict = tf.squeeze(model_outputs)


    valid_indices = (y_sp_test[7000:-900] != -1) & (y_ss_test[7000:-900] == True)
    err = y_sp_predict[7000:-900][valid_indices] - y_sp_test[7000:-900][valid_indices]


    rmse = np.sqrt(np.mean(np.square(err))) 
    rmses.append(rmse)


    err_isp = y_ss_predict[7000:-900]-y_ss_test[7000:-900]

    # Mask nan values
    err_isp = err_isp[~np.isnan(err_isp)]
    rmse_isp = np.sqrt(np.mean(np.square(err_isp))) 
    rmses_isp.append(rmse_isp)



    err_vel = y_v_predict - y_v_test
    err_vel = err_vel[y_v_test!=-1].numpy()
    rmse_vel = np.sqrt(np.mean(np.square(err_vel)))
    rmses_vel.append(rmse_vel)


    # Mask where y_r_test is not -100
    mask = y_r_test != -100

    # Apply mask to calculate RMSE
    err_ramp = y_r_predict[mask] - y_r_test[mask]
    rmse_ramp = np.sqrt(np.mean(np.square(err_ramp)))
    rmses_ramp.append(rmse_ramp)


mean_rmses_sp = np.mean(rmses)
mean_rmses_isp = np.mean(rmses_isp)
mean_rmses_vel = np.mean(rmses_vel)
mean_rmses_ramp = np.mean(rmses_ramp)

1/1 [==============================] - 0s 80ms/step


In [17]:
mean_rmses_sp

0.030393934

In [18]:
mean_rmses_isp

0.11119409

In [19]:
mean_rmses_vel

0.08286287

In [20]:
mean_rmses_ramp

1.0033383